In [10]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key as api_key

In [11]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [12]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
#Create a list for the cities
cities = []
#identify nearest city to lat and long
for coordinate in coordinates:
    city=citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

716

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID="+api_key

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_main_weather = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_main_weather})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | rikitea
Processing Record 2 of Set 1 | georgetown
Processing Record 3 of Set 1 | cherskiy
Processing Record 4 of Set 1 | misratah
Processing Record 5 of Set 1 | tura
Processing Record 6 of Set 1 | albany
Processing Record 7 of Set 1 | gogrial
Processing Record 8 of Set 1 | butaritari
Processing Record 9 of Set 1 | busselton
Processing Record 10 of Set 1 | tuktoyaktuk
Processing Record 11 of Set 1 | vaini
Processing Record 12 of Set 1 | bredasdorp
Processing Record 13 of Set 1 | treinta y tres
Processing Record 14 of Set 1 | torbay
Processing Record 15 of Set 1 | tulum
Processing Record 16 of Set 1 | san carlos de bariloche
Processing Record 17 of Set 1 | hofn
Processing Record 18 of Set 1 | kapaa
Processing Record 19 of Set 1 | provideniya
Processing Record 20 of Set 1 | ilulissat
Processing Record 21 of Set 1 | hermanus
Processing Record 22 of Set 1 | harper
Processing Record 23 of Set 1 | tarau

In [7]:
print(len(city_data))

652


In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Rikitea,PF,-23.1203,-134.9692,76.24,74,2,18.03,light rain
1,Georgetown,MY,5.4112,100.3354,79.90,87,40,3.44,scattered clouds
2,Cherskiy,RU,68.7500,161.3000,-34.65,100,62,4.07,broken clouds
3,Misratah,LY,32.3754,15.0925,57.24,52,0,13.49,clear sky
4,Tura,IN,25.5198,90.2201,72.70,54,10,2.39,clear sky
5,Albany,US,42.6001,-73.9662,35.31,81,100,4.23,overcast clouds
6,Gogrial,SS,8.5332,28.1004,76.48,65,82,4.41,broken clouds
7,Butaritari,KI,3.0707,172.7902,82.27,76,72,11.83,broken clouds
8,Busselton,AU,-33.6500,115.3333,72.09,36,6,4.76,clear sky
9,Tuktoyaktuk,CA,69.4541,-133.0374,-8.50,94,100,5.32,overcast clouds


In [9]:
#Create the output file CSV
output_file="WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_file, index_label="City_ID")